In [1]:
from reservoir_rls_multires import *
import matplotlib.pyplot as plt
from lorenz63 import *
from scipy.signal import welch, periodogram
from sklearn.preprocessing import StandardScaler
import cma

def phase_min_func(delay, truth, filtered):
    delay = round(delay)
    truth_delayed = truth[:truth.shape[0]-delay]
    filtered_delayed = filtered[delay:]
    val = -np.mean(truth_delayed * filtered_delayed)
    return val
def min_func_wphase(x, mask, base_data, f_s, true_external_data,\
    base_res, num_tests, num_nodes, pred_length, train_length, scale = True, 
    external_output = True, evenspace = False, returnall = False):
    init_delay = 2000
    max_freq = 0.15
    min_freq = 0.001
    Wn_xy = x[0]/10*(max_freq-min_freq)+min_freq
    Wn_z = Wn_xy
    data_split = separate_lorenz_2scale(base_data, f_s, Wn_xy, Wn_z, filter_order = 10)
    base_external_data = data_split[init_delay:,-1]
    """
    num_delays = 3000
    z_centered = base_external_data - np.mean(base_external_data)
    min_func   = lambda delay: phase_min_func(delay, true_external_data[init_delay:], z_centered)
    func_vals = np.zeros(num_delays)
    for i in range(num_delays):
        func_vals[i] = min_func(i)
    min_delay = np.argmin(func_vals)
    print(min_delay)
    """
    min_delay = 2460
    data = base_data[init_delay:base_data.shape[0]-min_delay]
    external_data = base_external_data[min_delay:]
    if scale:
        SS = StandardScaler()
        external_data = SS.fit_transform(external_data.reshape(-1,1))
    funval = vt_min_function_norm_external(data,external_data, x[1:], mask, base_res.Win, base_res.A, \
        num_tests = num_tests,  num_nodes = num_nodes, pred_length = pred_length, train_length = train_length,\
        external_output = external_output, evenspace = evenspace, returnall = returnall)
    return funval
def min_func_wtruth(x, mask, base_data, f_s, true_external_data,\
    base_res, num_tests, num_nodes, pred_length, train_length, scale = True, 
    external_output = True, evenspace = False, returnall = False):
    init_delay = 20000
    data = base_data[init_delay:]
    external_data = true_external_data[init_delay:]
    if scale:
        SS = StandardScaler()
        external_data = SS.fit_transform(external_data.reshape(-1,1))
    funval = vt_min_function_norm_external(data,external_data, x, mask, base_res.Win, base_res.A, \
        num_tests = num_tests,  num_nodes = num_nodes, pred_length = pred_length, train_length = train_length,\
        external_output = external_output, evenspace = evenspace, returnall = returnall)
    return funval

In [3]:
num_iters = 1500
num_tests = 20
results_data = np.zeros((num_iters, num_tests))
for i in range(num_iters):
    get_data = True
    data_length = 100000
    step = 0.05
    f_s = 1/step
    scale = 0.01
    slow_var = 48/28
    r_t = lambda x: r_t_const(x)
    dx_dt = lambda x, time, r_t: dxdt_lorenz_rossler(x, time, r_t, scale = scale, slow_var = slow_var)
    if get_data:
        lorenz_data_rossler = getCoupledLorenzData(data_length, r_t, dx_dt, sample_tau = step, seed = i)
        # np.savetxt('lorenz_data_rossler_step%0.2f_scale%0.2f.csv' \
        # %(step, scale), lorenz_data_rossler, delimiter = ',')
    else:
        lorenz_data_rossler = np.loadtxt('lorenz_data_rossler_step%0.2f_scale%0.2f.csv' \
                                         %(step, scale), delimiter = ',')
    # np.savetxt('rossler_rho.csv', lorenz_data_rossler[:,4], delimiter = ',')
    num_nodes = 360
    train_length = 3000
    sync_length = 200
    pred_length = 500
    res_seed = 1
    base_res = reservoir(4,num_nodes,input_weight = 1, spectral_radius = 1, \
                         seed = res_seed) #Generate a reservoir
    mask = ['input_weight', 'regularization', 'leakage', 'forget']
    x0 = np.array([0.4729354559136676, 5.114917368940299, \
                   2.147679481844106, 0.4531741728874831, 9.919738699208718])
    min_func_base = lambda x: min_func_wphase(x, mask, \
        np.ascontiguousarray(lorenz_data_rossler[:,:3]), f_s, lorenz_data_rossler[:,4],\
        base_res, num_tests, num_nodes, pred_length, train_length, external_output = True,\
        evenspace = True, returnall = True)
    results = min_func_base(x0)
    results_data[i] = results
    print(i)
    np.savetxt('rossler_external_filtered_results.csv', results_data, delimiter = ',')

[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
0
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
2
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
3
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
4
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
5
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
6
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 1

64
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
65
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
66
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
67
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
68
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
69
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
70
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
71
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209

129
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
130
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
131
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
132
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
133
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
134
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
135
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
136
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 1243

[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
194
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
195
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
196
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
197
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
198
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
199
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
200
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 1

258
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
259
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
260
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
261
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
262
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
263
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
264
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
265
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 1243

...

1223
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1224
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1225
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1226
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1227
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1228
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1229
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1230
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 116

1287
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1288
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1289
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1290
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1291
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1292
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1293
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1294
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 116

1351
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1352
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1353
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1354
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1355
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1356
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1357
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1358
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 116

1415
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1416
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1417
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1418
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1419
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1420
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1421
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1422
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 116

1479
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1480
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1481
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1482
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1483
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1484
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1485
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 11655, 12432, 13209, 13986, 14763]
1486
[0, 777, 1554, 2331, 3108, 3885, 4662, 5439, 6216, 6993, 7770, 8547, 9324, 10101, 10878, 116

In [2]:
print(cma.CMAOptions())

{'AdaptSigma': 'True  # or False or any CMAAdaptSigmaBase class e.g. CMAAdaptSigmaTPA, CMAAdaptSigmaCSA', 'CMA_active': 'True  # negative update, conducted after the original update', 'CMA_cmean': '1  # learning rate for the mean value', 'CMA_const_trace': 'False  # normalize trace, 1, True, "arithm", "geom", "aeig", "geig" are valid', 'CMA_diagonal': '0*100*N/popsize**0.5  # nb of iterations with diagonal covariance matrix, True for always', 'CMA_eigenmethod': 'np.linalg.eigh  # or cma.utilities.math.eig or pygsl.eigen.eigenvectors', 'CMA_elitist': 'False  #v or "initial" or True, elitism likely impairs global search performance', 'CMA_injections_threshold_keep_len': '1  #v keep length if Mahalanobis length is below the given relative threshold', 'CMA_mirrors': 'popsize < 6  # values <0.5 are interpreted as fraction, values >1 as numbers (rounded), otherwise about 0.16 is used', 'CMA_mirrormethod': '2  # 0=unconditional, 1=selective, 2=selective with delay', 'CMA_mu': 'None  # parents

In [3]:
def phase_min_func(delay, truth, filtered):
    delay = round(delay)
    truth_delayed = truth[:truth.shape[0]-delay]
    filtered_delayed = filtered[delay:]
    val = -np.mean(truth_delayed * filtered_delayed)
    return val
def min_func_wphase(x, mask, base_data, f_s, true_external_data,\
    base_res, num_tests, num_nodes, pred_length, train_lenght, scale = True, 
    external_output = True):
    init_delay = 20000
    max_freq = 0.15
    min_freq = 0.001
    Wn_xy = x[0]/10*(max_freq-min_freq)+min_freq
    Wn_z = Wn_xy
    data_split = separate_lorenz_2scale(base_data, f_s, Wn_xy, Wn_z, filter_order = 10)
    base_external_data = data_split[init_delay:,-1]
    num_delays = 10000
    z_centered = base_external_data - np.mean(base_external_data)
    min_func   = lambda delay: phase_min_func(delay, true_external_data[init_delay:], z_centered)
    func_vals = np.zeros(num_delays)
    for i in range(num_delays):
        func_vals[i] = min_func(i)
    min_delay = np.argmin(func_vals)
    data = base_data[init_delay:base_data.shape[0]-min_delay]
    external_data = base_external_data[min_delay:]
    if scale:
        SS = StandardScaler()
        external_data = SS.fit_transform(external_data.reshape(-1,1))
    funval = vt_min_function_norm_external(data,external_data, x[1:], mask, base_res.Win, base_res.A, \
        num_tests = num_tests,  num_nodes = num_nodes, pred_length = pred_length, train_length = train_length,\
        external_output = external_output)
    return funval
num_nodes = 360
num_tests = 200
train_length = 3000
sync_length = 500
pred_length = 500
res_seed = 1
base_res = reservoir(4,num_nodes,input_weight = 1, spectral_radius = 1, seed = res_seed) #Generate a reservoir
mask = ['input_weight', 'regularization', 'leakage', 'forget']
x0 = np.array([1,6,4,0,9])
min_func_base = lambda x: min_func_wphase(x, mask, np.ascontiguousarray(lorenz_data_rossler[:,:3]), f_s, lorenz_data_rossler[:,4],\
    base_res, num_tests, num_nodes, pred_length, train_length, external_output = False)
sigma = 2

In [4]:
opts = cma.CMAOptions()
opts.set('popsize',10*x0.size) # Set number of samples per generation
"""
Set bounds on parameters. IMPORTANT: The mean returned by cma-es is
the mean BEFORE the boundary function is applied, so the mean may not
lie in the domain set by bounds. To obtain the true sample mean requires 
downloading the cma-es package from github and editing one of the 
functions. Ask me if you need to do this.
"""
opts.set('bounds', [0,10]) 
opts.set('seed', 5) # Seed for the initial samples
"""
File where results are saved. IMPORTANT: Full covariance matrix is 
NOT saved, nor are the exact samples. If these need to be saved, one
will also have to download from github and make some edits. Again,
ask me.
"""
opts.set('verb_filenameprefix','cmaes_lorenz_rossler_wextern_scaled_res%d\\' % res_seed)
results = cma.fmin(min_func_base, x0, sigma, options = opts) # Run the algorithm

(25_w,50)-aCMA-ES (mu_w=14.0,w_1=14%) in dimension 5 (seed=5, Thu Jan 21 13:46:23 2021)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     50 -2.750000000000000e+01 1.0e+00 2.06e+00  2e+00  2e+00 29:41.3
    2    100 -2.800000000000000e+01 1.4e+00 2.01e+00  1e+00  2e+00 60:09.0
    3    150 -3.350000000000000e+01 2.2e+00 1.76e+00  8e-01  2e+00 90:31.7
    4    200 -3.300000000000000e+01 2.8e+00 1.93e+00  6e-01  2e+00 120:05.4
    5    250 -2.900000000000000e+01 3.5e+00 1.76e+00  5e-01  2e+00 149:38.8
    6    300 -3.300000000000000e+01 4.0e+00 1.62e+00  4e-01  1e+00 177:42.7
    7    350 -3.350000000000000e+01 4.8e+00 1.93e+00  4e-01  2e+00 205:41.9
    8    400 -3.600000000000000e+01 6.1e+00 1.82e+00  3e-01  2e+00 233:56.6
    9    450 -3.700000000000000e+01 7.4e+00 1.58e+00  2e-01  2e+00 261:59.9
   10    500 -3.750000000000000e+01 8.0e+00 1.46e+00  2e-01  1e+00 289:44.4
   11    550 -3.400000000000000e+01 8.2e+00 1.44e+00  2e-01  1e+00 317:21.6
   12  